# Spatial Features - finding imagery

Spatial features (spfeas) is a landcover classification tool focused on leveraging imagery textures for classifying landcover. It has been used in the World Bank for both quantifying informal housing and estimating poverty

Running spatial features is a 5 part process:

0. Setting up gbdx for spatial features
1. **Finding imagery**
2. Runnning spfeas
3. Downloading results from AWS
4. Stacking results
5. Running classification

### Finding appropriate imagery
Finding appropriate imagery for the analysis is a two part process. First, this script will search for imagery and generate a CSV of imagery options. Second, the csv and the AOI need to be compared in order to select the best coverage overlay (QGIS is best for this).

In [1]:
# Define input datasets
AOI = r"C:\Work\NPL_NYU\sample_3apr\sampleShapefile_3apr\sample.shp"
imageryCSV = AOI.replace(".shp", "_imagerySearch.csv")
### I don't know why these are required for the imagery search - I don't think they do anything, but it works, so just leave it
outFolder = "C:/temp"
searchName = "FUBAR"

cutOffDate = "1-Jan-05"  #earliest acceptable date
optimalDate = "30-Apr-19" #best date
maxAcceptableCloud = 100
curoffNadir = 90

In [2]:
import sys, os, inspect, importlib
import pandas as pd
import geopandas as gpd

from gbdxtools import Interface
from gbdxtools import CatalogImage
from shapely.geometry import box

cmd_folder = os.path.dirname(os.getcwd())
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from GOST_GBDx_Tools import imagery_search

#In order for the interface to be properly authenticated, follow instructions here:
#   http://gbdxtools.readthedocs.io/en/latest/user_guide.html
#   For Ben, the .gbdx-config file belongs in C:\Users\WB411133 (CAUSE no one else qill f%*$&ing tell you that)
gbdx = Interface()
gbdx.s3.info
curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)
importlib.reload(imagery_search)

<module 'GOST_GBDx_Tools.imagery_search' from 'C:\\Work\\Code\\GOST_GBDX\\GOST_GBDx_Tools\\imagery_search.py'>

In [5]:
inS = gpd.read_file(AOI)
inS = inS.to_crs({'init': u'epsg:4326'})
curWKT = inS['geometry']
for idx, row in inS.iterrows():
    imageryCSV = AOI.replace(".shp", "_imagerySearch_%s.csv" % idx)
    curWKT = row['geometry']
    curRes = imagery_search.searchForImages(gbdx, curWKT, outFolder, searchName, 
                    cutoff_date=cutOffDate, optimal_date=optimalDate,
                    cutoff_cloud_cover = maxAcceptableCloud, cutoff_nadir = curoffNadir,imageType='*')
    try:
        final = final.append(curRes)
    except:
        final = curRes

C:\WBG\Anaconda\envs\gbdxtools\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Work\Code\GOST_GBDX\GOST_GBDx_Tools\imagery_search.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  out_1stcut['points_CC'] = (out_1stcut['CloudCover'])
C:\Work\Code\GOST_GBDX\GOST_GBDx_Tools\imagery_search.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [17]:
imageryCSV = AOI.replace(".shp", "_imagerySearch.csv")
final2 = final[~final.duplicated(['ID'])]
final2.to_csv(imageryCSV)

In [16]:
final.shape

(1106, 25)